# Chapter 39: ctypes Advanced

This notebook covers advanced `ctypes` features including structures, unions, arrays,
and function pointer callbacks. These are the building blocks for defining complex C data
layouts and interfacing with C libraries that use structs, unions, and callback functions.

## Key Concepts
- **`Structure`**: Define C-compatible structs with `_fields_`
- **`Union`**: Overlapping fields that share memory
- **Arrays**: Fixed-size arrays using the `type * count` syntax
- **`CFUNCTYPE`**: Create C-callable function pointers from Python functions
- **Nested structures and pointers**: Compose complex C data layouts

## Section 1: Defining Structures

A `ctypes.Structure` subclass defines a C-compatible struct. The `_fields_` class
variable is a list of `(name, type)` tuples that define the struct layout. Fields
are accessible as attributes.

In [ ]:
import ctypes


class Point(ctypes.Structure):
    """A 2D point with integer coordinates."""
    _fields_ = [("x", ctypes.c_int), ("y", ctypes.c_int)]


# Create by positional arguments
p1: Point = Point(10, 20)
print(f"Point({p1.x}, {p1.y})")

# Create by keyword arguments
p2: Point = Point(x=30, y=40)
print(f"Point({p2.x}, {p2.y})")

# Modify fields
p1.x = 100
print(f"\nModified: Point({p1.x}, {p1.y})")

# Default values are zero
p3: Point = Point()
print(f"Default: Point({p3.x}, {p3.y})")

In [ ]:
import ctypes


class Pair(ctypes.Structure):
    """A pair of two integers."""
    _fields_ = [("a", ctypes.c_int), ("b", ctypes.c_int)]


# sizeof a structure matches C layout
pair: Pair = Pair(1, 2)
print(f"Pair({pair.a}, {pair.b})")
print(f"sizeof(Pair): {ctypes.sizeof(Pair)} bytes")
print(f"Expected (2 * c_int): {2 * ctypes.sizeof(ctypes.c_int)} bytes")
print(f"Match: {ctypes.sizeof(Pair) == 8}")

## Section 2: Structures with Mixed Types

Structures can contain fields of different types, including floats, doubles, and chars.
Be aware of potential padding bytes that the compiler may insert for alignment.

In [ ]:
import ctypes


class Particle(ctypes.Structure):
    """A particle with position and mass."""
    _fields_ = [
        ("x", ctypes.c_double),
        ("y", ctypes.c_double),
        ("mass", ctypes.c_float),
        ("active", ctypes.c_bool),
    ]


particle: Particle = Particle(x=1.5, y=2.5, mass=10.0, active=True)
print(f"Position: ({particle.x}, {particle.y})")
print(f"Mass: {particle.mass}")
print(f"Active: {particle.active}")
print(f"sizeof(Particle): {ctypes.sizeof(Particle)} bytes")

# Inspect the fields
print(f"\nFields:")
for name, field_type in Particle._fields_:
    print(f"  {name:10s} -> {field_type.__name__:15s} ({ctypes.sizeof(field_type)} bytes)")

In [ ]:
import ctypes


class Person(ctypes.Structure):
    """A person with name buffer and age."""
    _fields_ = [
        ("name", ctypes.c_char * 20),  # Fixed-size char array
        ("age", ctypes.c_int),
    ]


person: Person = Person(name=b"Alice", age=30)
print(f"Name: {person.name}")
print(f"Age: {person.age}")
print(f"sizeof(Person): {ctypes.sizeof(Person)} bytes")

# Update the name
person.name = b"Bob"
print(f"\nUpdated name: {person.name}")

## Section 3: ctypes Arrays

ctypes arrays are fixed-size sequences of a single type, created with the `type * count`
syntax. They behave like C arrays with a known length and support indexing.

In [ ]:
import ctypes

# Create an array type: 5 ints
IntArray5 = ctypes.c_int * 5

# Create an instance with values
arr: ctypes.Array[ctypes.c_int] = IntArray5(1, 2, 3, 4, 5)

print(f"Length: {len(arr)}")
print(f"First element: {arr[0]}")
print(f"Last element: {arr[4]}")

# Iterate over the array
print(f"\nAll elements:")
for i in range(len(arr)):
    print(f"  arr[{i}] = {arr[i]}")

# Convert to a Python list
as_list: list[int] = list(arr)
print(f"\nAs list: {as_list}")

In [ ]:
import ctypes

# Modify array elements
IntArray5 = ctypes.c_int * 5
arr: ctypes.Array[ctypes.c_int] = IntArray5(10, 20, 30, 40, 50)

arr[0] = 100
arr[4] = 500
print(f"Modified array: {list(arr)}")

# Double array
DoubleArray3 = ctypes.c_double * 3
darr: ctypes.Array[ctypes.c_double] = DoubleArray3(1.1, 2.2, 3.3)
print(f"\nDouble array: {list(darr)}")
print(f"sizeof(DoubleArray3): {ctypes.sizeof(DoubleArray3)} bytes")
print(f"Expected (3 * 8): {3 * ctypes.sizeof(ctypes.c_double)} bytes")

# Zero-initialized array
zeros: ctypes.Array[ctypes.c_int] = IntArray5()
print(f"\nZero-initialized: {list(zeros)}")

## Section 4: Unions

A `ctypes.Union` is like a structure, but all fields share the same memory location.
The size of a union equals the size of its largest field. Writing to one field
affects the raw bytes visible through all other fields.

In [ ]:
import ctypes


class IntOrFloat(ctypes.Union):
    """A union that can be interpreted as int or float."""
    _fields_ = [("i", ctypes.c_int), ("f", ctypes.c_float)]


# Set the integer field
u: IntOrFloat = IntOrFloat()
u.i = 0
print(f"i = 0 -> f = {u.f}")
print(f"Both zero: {u.f == 0.0}")

# The fields share memory, so setting one affects the other
u.i = 1065353216  # IEEE 754 representation of 1.0
print(f"\ni = 1065353216 -> f = {u.f}")

# Size equals the largest field
print(f"\nsizeof(IntOrFloat): {ctypes.sizeof(IntOrFloat)} bytes")
print(f"sizeof(c_int): {ctypes.sizeof(ctypes.c_int)} bytes")
print(f"sizeof(c_float): {ctypes.sizeof(ctypes.c_float)} bytes")

In [ ]:
import ctypes


class Number(ctypes.Union):
    """A union demonstrating different numeric interpretations."""
    _fields_ = [
        ("as_byte", ctypes.c_uint8),
        ("as_short", ctypes.c_uint16),
        ("as_int", ctypes.c_uint32),
    ]


n: Number = Number()
n.as_int = 0x01020304

print(f"as_int:   0x{n.as_int:08x}")
print(f"as_short: 0x{n.as_short:04x}")
print(f"as_byte:  0x{n.as_byte:02x}")
print(f"\nsizeof(Number): {ctypes.sizeof(Number)} bytes")
print(f"Union size matches largest field (c_uint32 = 4): {ctypes.sizeof(Number) == 4}")

## Section 5: CFUNCTYPE -- Function Pointer Callbacks

`CFUNCTYPE` creates a C-callable function type from a return type and argument types.
You can wrap a Python function in this type to create a callback that can be passed
to C libraries. The first argument is the return type, followed by parameter types.

In [ ]:
import ctypes

# Define a callback type: int callback(int, int)
CALLBACK = ctypes.CFUNCTYPE(ctypes.c_int, ctypes.c_int, ctypes.c_int)


def py_add(a: int, b: int) -> int:
    """A Python function to be used as a C callback."""
    return a + b


# Wrap the Python function
cb = CALLBACK(py_add)

# Call it -- it behaves like a C function pointer
result: int = cb(3, 4)
print(f"cb(3, 4) = {result}")
print(f"Result is 7: {result == 7}")

# Works with different arguments
print(f"cb(10, 20) = {cb(10, 20)}")
print(f"cb(-5, 5) = {cb(-5, 5)}")

In [ ]:
import ctypes

# Callback with different signatures

# double callback(double)
DoubleFunc = ctypes.CFUNCTYPE(ctypes.c_double, ctypes.c_double)


def py_square(x: float) -> float:
    """Return x squared."""
    return x * x


square_cb = DoubleFunc(py_square)
print(f"square(3.0) = {square_cb(3.0)}")
print(f"square(2.5) = {square_cb(2.5)}")

# Void return type (use None for void)
VoidFunc = ctypes.CFUNCTYPE(None, ctypes.c_int)


def py_print_int(x: int) -> None:
    """Print an integer (void return)."""
    print(f"  Callback received: {x}")


print_cb = VoidFunc(py_print_int)
print("\nCalling void callback:")
print_cb(42)
print_cb(99)

In [ ]:
import ctypes

# Practical example: using callbacks for a sort comparator
CompareFunc = ctypes.CFUNCTYPE(ctypes.c_int, ctypes.c_int, ctypes.c_int)


def ascending(a: int, b: int) -> int:
    """Compare for ascending order."""
    return a - b


def descending(a: int, b: int) -> int:
    """Compare for descending order."""
    return b - a


def apply_sort_key(
    values: list[int],
    compare: CompareFunc,
) -> list[int]:
    """Sort a list using a ctypes comparison callback."""
    import functools
    return sorted(values, key=functools.cmp_to_key(lambda a, b: compare(a, b)))


asc_cb = CompareFunc(ascending)
desc_cb = CompareFunc(descending)

data: list[int] = [5, 2, 8, 1, 9, 3]
print(f"Original:   {data}")
print(f"Ascending:  {apply_sort_key(data, asc_cb)}")
print(f"Descending: {apply_sort_key(data, desc_cb)}")

## Section 6: Nested Structures and Pointers to Structures

Structures can contain other structures as fields or pointers to structures.
This allows modeling complex C data layouts from Python.

In [ ]:
import ctypes


class Point(ctypes.Structure):
    """A 2D point."""
    _fields_ = [("x", ctypes.c_int), ("y", ctypes.c_int)]


class Rectangle(ctypes.Structure):
    """A rectangle defined by two corner points."""
    _fields_ = [("top_left", Point), ("bottom_right", Point)]


rect: Rectangle = Rectangle(
    top_left=Point(0, 0),
    bottom_right=Point(100, 50),
)

print(f"Top-left:     ({rect.top_left.x}, {rect.top_left.y})")
print(f"Bottom-right: ({rect.bottom_right.x}, {rect.bottom_right.y})")

# Calculate dimensions
width: int = rect.bottom_right.x - rect.top_left.x
height: int = rect.bottom_right.y - rect.top_left.y
print(f"\nWidth:  {width}")
print(f"Height: {height}")
print(f"sizeof(Rectangle): {ctypes.sizeof(Rectangle)} bytes")
print(f"Expected (4 ints * 4 bytes): {4 * ctypes.sizeof(ctypes.c_int)} bytes")

In [ ]:
import ctypes


class Point(ctypes.Structure):
    """A 2D point."""
    _fields_ = [("x", ctypes.c_int), ("y", ctypes.c_int)]


# Pointer to a structure
p: Point = Point(10, 20)
ptr = ctypes.pointer(p)

print(f"Original: ({p.x}, {p.y})")
print(f"Via pointer: ({ptr.contents.x}, {ptr.contents.y})")

# Modify through pointer
ptr.contents.x = 99
print(f"\nAfter pointer modification:")
print(f"Original p.x: {p.x}")
print(f"Pointer p.x:  {ptr.contents.x}")

# Structure containing an array field
class Vector3(ctypes.Structure):
    """A 3D vector using an array field."""
    _fields_ = [("coords", ctypes.c_double * 3)]


v: Vector3 = Vector3(coords=(ctypes.c_double * 3)(1.0, 2.0, 3.0))
print(f"\nVector3: ({v.coords[0]}, {v.coords[1]}, {v.coords[2]})")
print(f"sizeof(Vector3): {ctypes.sizeof(Vector3)} bytes")

## Section 7: Structures with Arrays

Combining structures and arrays allows defining complex data layouts that mirror
C struct definitions containing fixed-size array members.

In [ ]:
import ctypes


class DataPacket(ctypes.Structure):
    """A data packet with a header and fixed-size payload."""
    _fields_ = [
        ("packet_id", ctypes.c_uint32),
        ("length", ctypes.c_uint16),
        ("data", ctypes.c_uint8 * 8),
    ]


# Create a packet
payload = (ctypes.c_uint8 * 8)(0x48, 0x65, 0x6C, 0x6C, 0x6F, 0x00, 0x00, 0x00)
pkt: DataPacket = DataPacket(packet_id=1, length=5, data=payload)

print(f"Packet ID: {pkt.packet_id}")
print(f"Length: {pkt.length}")
print(f"Data bytes: {list(pkt.data)}")

# Interpret data as a string
data_bytes: bytes = bytes(pkt.data[:pkt.length])
print(f"Data as string: {data_bytes.decode('ascii')}")
print(f"\nsizeof(DataPacket): {ctypes.sizeof(DataPacket)} bytes")

## Summary

### Structures
- Subclass `ctypes.Structure` and define `_fields_` as `[(name, type), ...]`
- Access fields as attributes: `struct.field_name`
- `sizeof(StructType)` returns the total size including alignment padding
- Structures can be nested: use one Structure type as a field type in another

### Unions
- Subclass `ctypes.Union` with `_fields_` (same syntax as Structure)
- All fields share the same memory -- writing one field affects all others
- `sizeof(UnionType)` equals the size of the largest field

### Arrays
- Create array types with `type * count` (e.g., `c_int * 5`)
- Fixed length, indexable, iterable
- Can be used as fields in structures

### CFUNCTYPE Callbacks
- `CFUNCTYPE(restype, *argtypes)` defines a C-callable function type
- Wrap a Python function: `cb = CFUNCTYPE(...)(python_func)`
- Use `None` as return type for void functions
- Keep a reference to the callback to prevent garbage collection